**Task 07: Querying RDF(s)**

In [25]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [26]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N5d95faf6cbd2463b8c4cc0b8b6723b78 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [27]:
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
NS = Namespace("http://somewhere#")

for s, p, o in g.triples((None,RDFS.subClassOf,NS.Person)):
  print(s)
  

http://somewhere#Researcher


In [28]:
from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
SELECT ?subject
WHERE {
  ?subject rdfs:subClassOf ns:Person.
}
''',
initNs = {"rdfs":RDFS, "ns":NS}
)

for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [37]:
for s, p, o in g.triples((None, RDF.type, NS.Person)):
  print(s) 

for s,p,o in g.triples((None, RDFS.subClassOf, NS.Person)):
  for x,y,z in g.triples((None, RDF.type, s)):
    print(x)



http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith


In [40]:
q2 = prepareQuery('''
  SELECT ?Person
  WHERE { 
    {
    ?Person rdf:type ns:Person.
    } UNION {
      ?x rdfs:subClassOf ns:Person.
      ?Person rdf:type ?x
    } 
  }
  ''',
  initNs = { "ns": NS}
)

for r in g.query(q2):
  print(r)
  

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [43]:
for s, p, o in g.triples((None, RDF.type, NS.Person)):
  for a,b,c in g.triples((s, None, None)):
      print(a,b,c)

for s,p,o in g.triples((None, RDFS.subClassOf, NS.Person)):
  for a,b,c in g.triples((None, RDF.type, s)):
    for x,y,z in g.triples((a, None, None)):
      print(x,y,z)


http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere

In [48]:
q3 = prepareQuery('''
 SELECT ?Person ?Prop ?c
  WHERE {
   {
    ?Person rdf:type ns:Person.
    ?Person ?Prop ?c

  }UNION{ 
     ?Person rdf:type ?a.
     ?Person ?Prop ?c.
     ?a rdfs:subClassOf ns:Person
   }
  }
  ''',
  initNs = { "ns": ns, "rdf": RDF, "rdfs": RDFS}
)

for a in g.query(q3):
  print(a)

(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('Sara', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('Sara Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/